In [1]:
##предварительная обработка текста

In [ ]:
#!pip install numpy

In [1]:
import numpy as np
import pandas as pd
import re
import nltk
import string

In [4]:
#help(pd.read_csv)

In [3]:
##Чтение датасета с текстом
df = pd.read_csv("reviews.csv", encoding='UTF8', sep="\t")
df.head(10)

,review,sentiment
0,качество плохое пошив ужасный (горловина напер...,negative


In [6]:
df.iloc[[1]]
df.tail(3)

,review,sentiment
89997,спасибо большое ) продовца рекомендую.. заказа...,positive
89998,Очень довольна заказом! Меньше месяца в РБ. К...,positive
89999,хорошая куртка. постороннего запаха нет. швы р...,positive


In [8]:
##Нижний регистр. Например, Товар --> товар, УРА --> ура
df['text_lower']  = df['review'].str.lower()
df['text_lower'].head()

0    качество плохое пошив ужасный (горловина напер...
1    товар отдали другому человеку, я не получила п...
2    ужасная синтетика! тонкая, ничего общего с пре...
3    товар не пришел, продавец продлил защиту без м...
4        кофточка голая синтетика, носить не возможно.
Name: text_lower, dtype: object

In [9]:
df.head()

,review,sentiment,text_lower
0,качество плохое пошив ужасный (горловина напер...,negative,качество плохое пошив ужасный (горловина напер...
1,"Товар отдали другому человеку, я не получила п...",negative,"товар отдали другому человеку, я не получила п..."
2,"Ужасная синтетика! Тонкая, ничего общего с пре...",negative,"ужасная синтетика! тонкая, ничего общего с пре..."
3,"товар не пришел, продавец продлил защиту без м...",negative,"товар не пришел, продавец продлил защиту без м..."
4,"Кофточка голая синтетика, носить не возможно.",negative,"кофточка голая синтетика, носить не возможно."


In [11]:
#Удаление пунктуации
df['text_punct'] = df['review'].str.replace('[^\w\s]','')
df['text_punct'].head()

0    качество плохое пошив ужасный (горловина напер...
1    Товар отдали другому человеку, я не получила п...
2    Ужасная синтетика! Тонкая, ничего общего с пре...
3    товар не пришел, продавец продлил защиту без м...
4        Кофточка голая синтетика, носить не возможно.
Name: text_punct, dtype: object

In [6]:
#from nltk.corpus import stopwords
#print(stopwords.words('russian'))

In [12]:
#Удаление стоп-слов
#Импорт стоп-слов из бибилиотеки nltk
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('russian'))
# функция, удаляющая стопслова из текстов

def stopwords(text):
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])
# Применим функцию удаления стоп-слов
df["text_stop"] = df["text_punct"].apply(stopwords)
df["text_stop"].head()

0    качество плохое пошив ужасный (горловина напер...
1    Товар отдали другому человеку, получила посылк...
2    Ужасная синтетика! Тонкая, общего представленн...
3    товар пришел, продавец продлил защиту моего со...
4           Кофточка голая синтетика, носить возможно.
Name: text_stop, dtype: object

In [13]:
#Удаление высокочастотных слов
from collections import Counter
cnt = Counter()
for text in df["text_stop"].values:
    for word in text.split():
        cnt[word] += 1
        
cnt.most_common(10)

[('очень', 21054),
 ('размер', 13953),
 (',', 7945),
 ('деньги', 7917),
 ('продавец', 7195),
 ('На', 7174),
 ('это', 6822),
 ('товар', 6755),
 ('-', 6536),
 ('качество', 6499)]

In [14]:
#ужаление часто встречающиеся слов в корпусе при помощи tf-idf
# Удаление самых частотных слов
freq = set([w for (w, wc) in cnt.most_common(10)])
# функция удаления слов
def freqwords(text):
    return " ".join([word for word in str(text).split() if word not 
in freq])
# применение функции
df["text_common"] = df["text_stop"].apply(freqwords)
df["text_common"].head()

0    плохое пошив ужасный (горловина наперекос) Фот...
1    Товар отдали другому человеку, получила посылк...
2    Ужасная синтетика! Тонкая, общего представленн...
3    пришел, продлил защиту моего согласия. продавц...
4           Кофточка голая синтетика, носить возможно.
Name: text_common, dtype: object

In [15]:
#Удаление низкочастотных слов
freq = pd.Series(' '.join(df['text_common']).split()).value_counts()[-10:] # 10 rare words
freq = list(freq.index)
df['text_rare'] = df['text_common'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
df['text_rare'].head()

0    плохое пошив ужасный (горловина наперекос) Фот...
1    Товар отдали другому человеку, получила посылк...
2    Ужасная синтетика! Тонкая, общего представленн...
3    пришел, продлил защиту моего согласия. продавц...
4           Кофточка голая синтетика, носить возможно.
Name: text_rare, dtype: object

In [16]:
# Удаление эмодзи
def emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)
#passing the emoji function to 'text_rare'
df['text_rare'] = df['text_rare'].apply(emoji)

In [17]:
#Удаление цифр
df['text_nonum'] = df['text_common'].str.replace('\d+', '') 
df['text_nonum'].head()

0    плохое пошив ужасный (горловина наперекос) Фот...
1    Товар отдали другому человеку, получила посылк...
2    Ужасная синтетика! Тонкая, общего представленн...
3    пришел, продлил защиту моего согласия. продавц...
4           Кофточка голая синтетика, носить возможно.
Name: text_nonum, dtype: object

In [18]:
#Удаление URL
# Function for url's
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)
# Examples
text = "This is my website, https://www.link.com"
remove_urls(text)
#Passing the function to 'text_rare'
df['text_rare'] = df['text_rare'].apply(remove_urls)

In [19]:
#Удаление HTML-тегов
from bs4 import BeautifulSoup
#Function for removing html
def html(text):
    return BeautifulSoup(text, "lxml").text
# Examples
text = """<div>
<h1> Это </h1>
<p> странца</p>
<a href="https://www.link.com/"> пример</a>
</div>
"""
print(html(text))
# Passing the function to 'text_rare'
df['text_rare'] = df['text_rare'].apply(html)


 Это 
 странца
 пример




C:\ProgramData\anaconda3\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [20]:
#Токенизация
def tokenization(text):
    text = re.split('\W+', text)
    return text
# Passing the function to 'text_rare' and store into'text_token'
df['text_token'] = df['text_rare'].apply(lambda x: tokenization(x.lower()))
df[['text_token']].head()

,text_token
0,"[плохое, пошив, ужасный, горловина, наперекос,..."
1,"[товар, отдали, другому, человеку, получила, п..."
2,"[ужасная, синтетика, тонкая, общего, представл..."
3,"[пришел, продлил, защиту, моего, согласия, про..."
4,"[кофточка, голая, синтетика, носить, возможно, ]"


In [21]:
#Можно выполнить при помощи NLTK. Заодно проведем лемматизацию
from pymorphy2 import MorphAnalyzer
from nltk import sent_tokenize, word_tokenize, regexp_tokenize

def tokenize_lemmas(sent, pat=r"(?u)\b\w\w+\b", morph=MorphAnalyzer()):
    return [morph.parse(tok)[0].normal_form 
            for tok in regexp_tokenize(sent, pat)]
df["text_lemm"] = df["text_rare"].map(lambda x: " ".join(tokenize_lemmas(x)))
df[['text_lemm']].head()

,text_lemm
0,плохой пошив ужасный горловина наперекос фото ...
1,товар отдать другой человек получить посылка л...
2,ужасный синтетик тонкий общий представить карт...
3,прийти продлить защита мой согласие продавец о...
4,кофточка голый синтетик носить возможно


In [46]:
#Исходя из цели автоматической обработки текста, выберем необходимые варианты предобработки текста. 
#Например, для классификации можно воспользоваться нижним регистром, удалением  пунктуации, ссылок, стоп-слов, 
#произвести лемматизацию и токенизацию
##ИЛИ воспользоваться только удалением ссылок. Т.к. это актуальнее для НС, а выбор предобработки сделал для Машинного обученя
df['text_ready']  = df['review'].str.lower()
df['text_ready'] = df['text_ready'].str.replace('\d+', '') 
df['text_ready'] = df['text_ready'].str.replace('[^\w\s]','')
df["text_ready"] = df["text_ready"].apply(stopwords)
df['text_ready'] = df['text_ready'].apply(emoji)
df['text_ready'] = df['text_ready'].apply(remove_urls)
df['text_ready'] = df['text_ready'].apply(html)
df["text_ready"] = df["text_ready"].map(lambda x: " ".join(tokenize_lemmas(x)))
df[['text_ready']].head()

,text_ready
0,качество плохой пошив ужасный горловина напере...
1,товар отдать другой человек получить посылка л...
2,ужасный синтетик тонкий общий представить карт...
3,товар прийти продавец продлить защита мой согл...
4,кофточка голый синтетик носить возможно


In [ ]:
##Пример препобученной модели HuggingFace Torch

In [17]:
import torch
from transformers import AutoModelForSequenceClassification
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained('blanchefort/rubert-base-cased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('blanchefort/rubert-base-cased-sentiment', return_dict=True)


def predict(text):
    inputs = tokenizer(text, max_length=512, padding=True, truncation=True, return_tensors='pt')
    outputs = model(**inputs)
    predicted = torch.nn.functional.softmax(outputs.logits, dim=1)
    predicted = torch.argmax(predicted, dim=1).numpy()
    return predicted

In [23]:
#txt = df.iloc[2,2]
txt = "Хорошая погода! Наконец-то тепло!"
print(txt)

Хорошая погода! Наконец-то тепло!


In [24]:
def predict(text):
    inputs = tokenizer(text, max_length=512, padding=True, truncation=True, return_tensors='pt')
    outputs = model(**inputs)
    predicted = torch.nn.functional.softmax(outputs.logits, dim=1)
    predicted = torch.argmax(predicted, dim=1).numpy()
    return predicted

In [25]:
 predict(txt)

array([1], dtype=int64)